In [21]:
# !pip install webdriver-manager

In [22]:
from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By   # para usar o find_element ou find_elements(lista de itens)
from selenium.webdriver.common.keys import Keys

# servico = Service(ChromeDriverManager().install())
nav = webdriver.Chrome(service=servico)

In [23]:
# pesquisar cotação dolar
nav.get("https://www.google.com/")
nav.find_element(By.XPATH,'//*[@id="APjFqb"]').send_keys("cotação dólar") # escrever na barra de pesquisa do google
nav.find_element(By.XPATH,'//*[@id="APjFqb"]').send_keys(Keys.ENTER) # dar enter na pesquisa
cotacao_dolar = nav.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

# pesquisar cotação euro
nav.get("https://www.google.com/")
nav.find_element(By.XPATH,'//*[@id="APjFqb"]').send_keys("cotação euro")
nav.find_element(By.XPATH,'//*[@id="APjFqb"]').send_keys(Keys.ENTER)
cotacao_euro = nav.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# pegar cotacao ouro
nav.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = nav.find_element(By.ID,'comercial').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

nav.quit()

4.8692
5.351177762000001
304.86


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [24]:
import pandas as pd

produtos_df = pd.read_excel("Produtos.xlsx")
display(produtos_df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [25]:
produtos_df.loc[produtos_df['Moeda']=="Dólar", "Cotação"] = float(cotacao_dolar)
produtos_df.loc[produtos_df['Moeda']=="Euro", "Cotação"] = float(cotacao_euro)
produtos_df.loc[produtos_df['Moeda']=="Ouro", "Cotação"] = float(cotacao_ouro)

produtos_df['Preço Base Reais'] = produtos_df['Cotação'] * produtos_df['Preço Base Original']
produtos_df['Preço Final'] = produtos_df['Ajuste'] * produtos_df['Preço Base Reais']
# produtos_df["Preço Final"] = produtos_df["Preço Final"].map("{:.2f}".format)
display(produtos_df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,4.869200,4869.151308,1.40,6816.811831
1,Carro Renault,4500.00,Euro,5.351178,24080.299929,2.00,48160.599858
2,Notebook Dell,899.99,Dólar,4.869200,4382.231308,1.70,7449.793224
3,IPhone,799.00,Dólar,4.869200,3890.490800,1.70,6613.834360
4,Carro Fiat,3000.00,Euro,5.351178,16053.533286,1.90,30501.713243
5,Celular Xiaomi,480.48,Dólar,4.869200,2339.553216,2.00,4679.106432
6,Joia 20g,20.00,Ouro,304.860000,6097.200000,1.15,7011.780000


### Agora vamos exportar a nova base de preços atualizada

In [26]:
produtos_df.to_excel("Produtos Atualizado.xlsx", index=False)